**Baseline** **Model**

In [1]:
%load_ext tensorboard

In [2]:
%%capture
!pip install datasets
!pip install tokenizers
!pip install torchmetrics

In [3]:
!git clone https://github.com/Irfan-Hamid/test11.git

Cloning into 'test11'...
remote: Enumerating objects: 215, done.
remote: Counting objects: 100% (102/102), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 215 (delta 59), reused 67 (delta 31), pack-reused 113
Receiving objects: 100% (215/215), 2.77 MiB | 5.13 MiB/s, done.
Resolving deltas: 100% (124/124), done.


In [4]:
%%capture
%cd /content/test11

In [5]:
!git branch -a

* main
  remotes/origin/HEAD -> origin/main
  remotes/origin/Irfan-Hamid-patch-1
  remotes/origin/final_check
  remotes/origin/main


In [6]:
!git checkout final_check

Branch 'final_check' set up to track remote branch 'final_check' from 'origin'.
Switched to a new branch 'final_check'


In [7]:
!git branch


* final_check
  main


In [8]:
!mkdir -p /content/Models/pytorch-transformer/weights
!mkdir -p /content/Models/pytorch-transformer/vocab

# !mkdir -p /content/drive/MyDrive/Models/pytorch-transformer/weights
# !mkdir -p /content/drive/MyDrive/Models/pytorch-transformer/vocab

In [9]:
!pip install nltk
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import corpus_bleu
!pip install nltk rouge jiwer
from nltk.translate.meteor_score import meteor_score
nltk.download('wordnet')
from rouge import Rouge
from jiwer import cer, wer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 50.6 MB/s eta 0:00:00


[nltk_data] Downloading package wordnet to /root/nltk_data...


In [10]:
import matplotlib.pyplot as plt

In [11]:
from config import get_config
cfg = get_config()
cfg['model_folder'] = '/content/Models/pytorch-transformer/weights'
cfg['tokenizer_file'] = '/content/Models/pytorch-transformer/vocab/tokenizer_{0}.json'
cfg['batch_size'] = 64
cfg['num_epochs'] = 50
cfg['preload'] = None

In [12]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 2.7 MB/s eta 0:00:00


In [13]:
from train import train_model

train_model(cfg)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet_ic to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet_ic.zip.


Using device: cuda
Device name: NVIDIA A100-SXM4-40GB
Device memory: 39.56427001953125 GB


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1404 [00:00<?, ? examples/s]

Max length of source sentence: 204
Max length of target sentence: 196
No model to preload, starting from scratch


Processing Epoch 49: 100%|██████████| 20/20 [00:12<00:00,  1.63it/s, loss=1.623]


BLEU Score: 22.61


In [14]:
import torch
import torch.nn as nn
from model import Transformer
from config import get_config, get_weights_file_path
from train import get_model, get_ds, greedy_decode
import altair as alt
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [15]:
# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [16]:
config = get_config()
train_dataloader, val_dataloader, vocab_src, vocab_tgt = get_ds(config)
model = get_model(config, vocab_src.get_vocab_size(), vocab_tgt.get_vocab_size()).to(device)

# Load the pretrained weights
config = get_config()
# Manually adjust the path for the specific runtime case
model_filename = "/content/Models/pytorch-transformer/weights/tmodel_49.pt"

# Load the pretrained weights
state = torch.load(model_filename)
model.load_state_dict(state['model_state_dict'])


Max length of source sentence: 204
Max length of target sentence: 196


<All keys matched successfully>

In [17]:
!ls /content/Models/pytorch-transformer/weights


tmodel_00.pt  tmodel_08.pt  tmodel_16.pt  tmodel_24.pt	tmodel_32.pt  tmodel_40.pt  tmodel_48.pt
tmodel_01.pt  tmodel_09.pt  tmodel_17.pt  tmodel_25.pt	tmodel_33.pt  tmodel_41.pt  tmodel_49.pt
tmodel_02.pt  tmodel_10.pt  tmodel_18.pt  tmodel_26.pt	tmodel_34.pt  tmodel_42.pt
tmodel_03.pt  tmodel_11.pt  tmodel_19.pt  tmodel_27.pt	tmodel_35.pt  tmodel_43.pt
tmodel_04.pt  tmodel_12.pt  tmodel_20.pt  tmodel_28.pt	tmodel_36.pt  tmodel_44.pt
tmodel_05.pt  tmodel_13.pt  tmodel_21.pt  tmodel_29.pt	tmodel_37.pt  tmodel_45.pt
tmodel_06.pt  tmodel_14.pt  tmodel_22.pt  tmodel_30.pt	tmodel_38.pt  tmodel_46.pt
tmodel_07.pt  tmodel_15.pt  tmodel_23.pt  tmodel_31.pt	tmodel_39.pt  tmodel_47.pt


In [40]:
def load_next_batch():
    # Load a sample batch from the validation set
    batch = next(iter(val_dataloader))
    encoder_input = batch["encoder_input"].to(device)
    encoder_mask = batch["encoder_mask"].to(device)
    decoder_input = batch["decoder_input"].to(device)
    decoder_mask = batch["decoder_mask"].to(device)

    encoder_input_tokens = [vocab_src.id_to_token(idx) for idx in encoder_input[0].cpu().numpy()]
    decoder_input_tokens = [vocab_tgt.id_to_token(idx) for idx in decoder_input[0].cpu().numpy()]

    # check that the batch size is 1
    assert encoder_input.size(
        0) == 1, "Batch"

    model_out = greedy_decode(
        model, encoder_input, encoder_mask, vocab_src, vocab_tgt, config['seq_len'], device)

    return batch, encoder_input_tokens, decoder_input_tokens

In [41]:
def mtx2df(m, max_row, max_col, row_tokens, col_tokens):
    return pd.DataFrame(
        [
            (
                r,
                c,
                float(m[r, c]),
                "%.3d %s" % (r, row_tokens[r] if len(row_tokens) > r else "<blank>"),
                "%.3d %s" % (c, col_tokens[c] if len(col_tokens) > c else "<blank>"),
            )
            for r in range(m.shape[0])
            for c in range(m.shape[1])
            if r < max_row and c < max_col
        ],
        columns=["row", "column", "value", "row_token", "col_token"],
    )

def get_attn_map(attn_type: str, layer: int, head: int):
    if attn_type == "encoder":
        attn = model.encoder.layers[layer].self_attention_block.attention_scores
    elif attn_type == "decoder":
        attn = model.decoder.layers[layer].self_attention_block.attention_scores
    elif attn_type == "encoder-decoder":
        attn = model.decoder.layers[layer].cross_attention_block.attention_scores
    return attn[0, head].data

def attn_map(attn_type, layer, head, row_tokens, col_tokens, max_sentence_len):
    df = mtx2df(
        get_attn_map(attn_type, layer, head),
        max_sentence_len,
        max_sentence_len,
        row_tokens,
        col_tokens,
    )
    return (
        alt.Chart(data=df)
        .mark_rect()
        .encode(
            x=alt.X("col_token", axis=alt.Axis(title="")),
            y=alt.Y("row_token", axis=alt.Axis(title="")),
            color="value",
            tooltip=["row", "column", "value", "row_token", "col_token"],
        )
        #.title(f"Layer {layer} Head {head}")
        .properties(height=400, width=400, title=f"Layer {layer} Head {head}")
        .interactive()
    )

def get_all_attention_maps(attn_type: str, layers: list[int], heads: list[int], row_tokens: list, col_tokens, max_sentence_len: int):
    charts = []
    for layer in layers:
        rowCharts = []
        for head in heads:
            rowCharts.append(attn_map(attn_type, layer, head, row_tokens, col_tokens, max_sentence_len))
        charts.append(alt.hconcat(*rowCharts))
    return alt.vconcat(*charts)

In [86]:
batch, encoder_input_tokens, decoder_input_tokens = load_next_batch()
print(f'Source: {batch["src_text"][0]}')
print(f'Target: {batch["tgt_text"][0]}')
sentence_len = encoder_input_tokens.index("[PAD]")

Source: However, everything is queer to-day.'
Target: Entretanto, tudo está estranho hoje!"


In [87]:
layers = [0, 1, 2]
heads = [0, 1, 2, 3, 4, 5, 6, 7]

# Encoder Self-Attention
get_all_attention_maps("decoder", layers, heads, decoder_input_tokens, decoder_input_tokens, min(20, sentence_len))


alt.VConcatChart(...)